In [2]:
import tensorflow as tf
import pandas as pd
from datasets import load_dataset

In [3]:
dataset  = load_dataset('banking77')

Using custom data configuration default
Reusing dataset banking77 (C:\Users\barka\.cache\huggingface\datasets\banking77\default\1.1.0\aec0289529599d4572d76ab00c8944cb84f88410ad0c9e7da26189d31f62a55b)


  0%|          | 0/2 [00:00<?, ?it/s]

In [4]:
train_df = pd.DataFrame(dataset['train'])
test_df = pd.DataFrame( dataset['test'])

In [56]:
AUTOTUNE = tf.data.AUTOTUNE

train_ds = tf.data.Dataset.from_tensor_slices((train_df['text'],train_df['label'])).shuffle(10000,seed=42).batch(32).cache().prefetch(buffer_size=AUTOTUNE)
test_ds = tf.data.Dataset.from_tensor_slices((test_df['text'],test_df['label'])).batch(32).cache().prefetch(buffer_size=AUTOTUNE,)

In [57]:
text_batch_1, text_label_1 = next(iter(train_ds)) # pick first batch from dataset
text , label = text_batch_1[0],text_label_1[0] # taking first text and label from batch
text , label

(<tf.Tensor: shape=(), dtype=string, numpy=b'Is it possible to do auto top up?'>,
 <tf.Tensor: shape=(), dtype=int64, numpy=4>)

In [27]:
import tensorflow_text as text  # Registers the ops.

In [121]:
import tensorflow_hub as hub
text_input = tf.keras.Input(shape=(),dtype=tf.string,name="input_layer")
preprocessor = hub.KerasLayer('https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',name="preprocessor")
encoder_inputs = preprocessor(text_input)

encoder = hub.KerasLayer('https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-2_H-128_A-2/2',trainable=True,name='BERT_encoder')
outputs = encoder(encoder_inputs)

pooled_output = outputs['pooled_output']

#create Dense layer for classifier
# net = tf.keras.layers.Dropout(0.1)(pooled_output)
net = tf.keras.layers.Dense(77,activation='softmax')(pooled_output)




In [122]:
embedding_model = tf.keras.Model(text_input, net)
sentences = tf.constant(["I am still waiting on my card?"])
print(embedding_model(sentences))

tf.Tensor(
[[0.00359659 0.01817336 0.01173988 0.00805034 0.00409168 0.00387391
  0.0150459  0.00899213 0.01174174 0.01080564 0.01971558 0.01027506
  0.00856184 0.00663818 0.01563069 0.01973611 0.0068118  0.00235368
  0.00490017 0.00662774 0.00757913 0.00782121 0.02019623 0.0630206
  0.00791129 0.00752038 0.00617793 0.02089857 0.00384475 0.0050759
  0.00865544 0.00548097 0.00617788 0.00564687 0.01898446 0.01697628
  0.0195923  0.03136914 0.01377622 0.00125803 0.05537669 0.01683473
  0.00647143 0.02611497 0.00901326 0.00464004 0.01293755 0.00409844
  0.01169057 0.03083093 0.01031169 0.01477323 0.00233885 0.07367555
  0.00667026 0.01010882 0.00419484 0.00455484 0.00215192 0.01577845
  0.01606135 0.00954381 0.03884484 0.00440478 0.00185708 0.01032211
  0.01201894 0.002648   0.00859236 0.00925962 0.00473721 0.00756066
  0.00653763 0.00650888 0.01444102 0.01706394 0.03170508]], shape=(1, 77), dtype=float32)


In [38]:
embedding_model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_layer (InputLayer)       [(None,)]            0           []                               
                                                                                                  
 preprocessor (KerasLayer)      {'input_type_ids':   0           ['input_layer[0][0]']            
                                (None, 128),                                                      
                                 'input_word_ids':                                                
                                (None, 128),                                                      
                                 'input_mask': (Non                                               
                                e, 128)}                                                    

In [43]:
TOTAL_SIZE = train_df.shape[0]
BATCH_SIZE = 32
VAL_BATCH  = int(TOTAL_SIZE / BATCH_SIZE) 

In [58]:
train_ds = train_ds.take(VAL_BATCH)
val_ds = train_ds.skip(VAL_BATCH)

In [ ]:
epochs = 5
# steps_per_epoch = tf.data.experimental.cardinality(train_ds).numpy()
# num_train_steps = steps_per_epoch * epochs
# num_warmup_steps = int(0.1*num_train_steps)

# init_lr = 3e-5
# optimizer = tf.keras.optimization.create_optimizer(init_lr=init_lr,
#                                           num_train_steps=num_train_steps,
#                                           num_warmup_steps=num_warmup_steps,
#                                           optimizer_type='adamw')

In [123]:
embedding_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5),
                         loss=tf.keras.losses.SparseCategoricalCrossentropy(),
                         metrics=['accuracy'])


In [124]:
embedding_model.fit(train_ds,validation_data=val_ds,epochs=5)

Epoch 1/5
312/312 [==============================] - 77s 239ms/step - loss: 4.5185 - accuracy: 0.0135
Epoch 2/5
312/312 [==============================] - 72s 230ms/step - loss: 4.2491 - accuracy: 0.0438
Epoch 3/5
312/312 [==============================] - 70s 223ms/step - loss: 4.0008 - accuracy: 0.1015
Epoch 4/5
312/312 [==============================] - 72s 230ms/step - loss: 3.7765 - accuracy: 0.1559
Epoch 5/5
312/312 [==============================] - 76s 242ms/step - loss: 3.5624 - accuracy: 0.2156


In [61]:
loss, accuracy = embedding_model.evaluate(test_ds)

97/97 [==============================] - 7s 64ms/step - loss: 3.5177 - accuracy: 0.2390


In [93]:
preds = embedding_model.predict(test_ds)

97/97 [==============================] - 6s 60ms/step


In [104]:
v = test_ds.take(1)

In [111]:
for x,y in train_ds.take(1):
    print(x,y)
    # break

tf.Tensor(
[b'Can a credit card be used to transfer money?'
 b'My card activation is failing.' b'How long will my top-up be pending?'
 b"Why hasn't my son received the transfer I sent to him earlier?"
 b'Will money be added automatically when necessary?'
 b"Do you not accept crypto transfers from certain places? Because when I tried to do a top up the money just disappeared. Im not sure what to do or what I'm doing wrong, please help."
 b'The wrong exchange rate applied to my cash withdrawal.'
 b"You promised no fees but now I've got one. What the hell!?"
 b'Why did I get charged a fee for my transfer?'
 b'How much is it to send out additional cards?'
 b'Hey I am standing in front of an ATM here, it only gave me 10 pounds even though I wanted to withdraw 30! Seems like the app has 30 pounds, what do I do??'
 b'Please give me both Visa and MasterCard, is it possible?'
 b'Why have I been charged twice?' b'I need my virtual card'
 b'I was charged too many times for one transaction.'
 b'Wh

In [105]:
embedding_model.predict(v)

1/1 [==============================] - 0s 77ms/step


array([[0.06842163, 0.00334039, 0.00698865, ..., 0.00877284, 0.00494816,
        0.00503086],
       [0.03302078, 0.00379312, 0.0133736 , ..., 0.00748981, 0.02237591,
        0.01538504],
       [0.05113184, 0.00421031, 0.00963745, ..., 0.00564119, 0.02267505,
        0.00645326],
       ...,
       [0.04194529, 0.00361456, 0.00887997, ..., 0.01052712, 0.0054382 ,
        0.00625578],
       [0.02375717, 0.00542869, 0.01179141, ..., 0.00801594, 0.02415427,
        0.01230364],
       [0.01862687, 0.00712863, 0.01008906, ..., 0.00598353, 0.02096801,
        0.0128925 ]], dtype=float32)

In [94]:
preds = tf.math.argmax(preds,1)

In [110]:
preds[:32]

<tf.Tensor: shape=(32,), dtype=int64, numpy=
array([ 0, 53, 53, 36, 53,  0, 47,  0, 15, 47, 21, 52, 45, 45, 47, 45, 30,
       53, 47, 30, 30,  0, 53, 30, 63, 15, 30, 30, 30, 58, 53, 30],
      dtype=int64)>

In [87]:
for x,_ in test_ds.take(1):
    print(x,_)

tf.Tensor(
[b'How do I locate my card?'
 b'I still have not received my new card, I ordered over a week ago.'
 b'I ordered a card but it has not arrived. Help please!'
 b'Is there a way to know when my card will arrive?'
 b'My card has not arrived yet.' b'When will I get my card?'
 b'Do you know if there is a tracking number for the new card you sent me?'
 b'i have not received my card' b'still waiting on that card'
 b'Is it normal to have to wait over a week for my new card?'
 b'How do I track my card?' b'How long does a card delivery take?'
 b"I still don't have my card after 2 weeks.  What should I do?"
 b'still waiting on my new card'
 b'I am still waiting for my card after 1 week.  Is this ok?'
 b'I have been waiting longer than expected for my bank card, could you provide information on when it will arrive?'
 b"I've been waiting longer than expected for my card."
 b"Why hasn't my card been delivered?"
 b'Where is my new card? I have been waiting a week!'
 b"My card still hasn't a